In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize a SparkSession
spark = SparkSession.builder.config("spark.jars", "postgresql-42.6.0.jar").master('local[4]').getOrCreate()

# slide 13

In [3]:
# Some data with their column names. With DF we can structure our data
columns = ["id","name","surname","age","country","local_phone"]
input_data = [(1,"Simón","Bolivar",47,"VEN","489 895 965"),
    (2,"Fidel","Castro",90,"CU","956 268 348"),
    (3,"Jose","Doroteo",45,"MEX","985 621 444"),
    (4,"Ernesto","Guevara",39,"AR","895 325 481"),
    (5,"Hugo","Chávez",58,"VE","489 895 965"),
    (6,"Camilo","Cienfuegos",27,"CUB","956 268 348"),
    (7,"Emiliano","Zapata",39,"ME","985 621 444"),
    (8,"Juan Domingo","Perón",78,"ARG","985 621 444"),
  ]

# Simplier data
int_list = [1,2,3]


# intDF = spark.createDataFrame(int_list).toDF("value") # this doesnt work
# DF from primitive indicating type
print("Easy DF")
intDF = spark.createDataFrame(int_list, "int").toDF("value")
intDF.printSchema()
intDF.show()

print("Complex DF")
complexDF = spark.createDataFrame(input_data)
complexDF.printSchema()
complexDF.show()

# slide 14

Easy DF
root
 |-- value: integer (nullable = true)



23/12/23 09:01:02 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/12/23 09:01:02 INFO DAGScheduler: Got job 4 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/12/23 09:01:02 INFO DAGScheduler: Final stage: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0)
23/12/23 09:01:02 INFO DAGScheduler: Parents of final stage: List()
23/12/23 09:01:02 INFO DAGScheduler: Missing parents: List()
23/12/23 09:01:02 INFO DAGScheduler: Submitting ResultStage 4 (MapPartitionsRDD[20] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/12/23 09:01:02 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 12.2 KiB, free 434.3 MiB)
23/12/23 09:01:02 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 6.5 KiB, free 434.3 MiB)
23/12/23 09:01:02 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 193.147.50.16:42587 (size: 6.5 KiB, free: 434.4 

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
+-----+

Complex DF
root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: long (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)

+---+------------+----------+---+---+-----------+
| _1|          _2|        _3| _4| _5|         _6|
+---+------------+----------+---+---+-----------+
|  1|       Simón|   Bolivar| 47|VEN|489 895 965|
|  2|       Fidel|    Castro| 90| CU|956 268 348|
|  3|        Jose|   Doroteo| 45|MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39| AR|895 325 481|
|  5|        Hugo|    Chávez| 58| VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|CUB|956 268 348|
|  7|    Emiliano|    Zapata| 39| ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|ARG|985 621 444|
+---+------------+----------+---+---+-----------+



23/12/23 09:01:03 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/12/23 09:01:03 INFO DAGScheduler: Got job 7 (showString at NativeMethodAccessorImpl.java:0) with 3 output partitions
23/12/23 09:01:03 INFO DAGScheduler: Final stage: ResultStage 7 (showString at NativeMethodAccessorImpl.java:0)
23/12/23 09:01:03 INFO DAGScheduler: Parents of final stage: List()
23/12/23 09:01:03 INFO DAGScheduler: Missing parents: List()
23/12/23 09:01:03 INFO DAGScheduler: Submitting ResultStage 7 (MapPartitionsRDD[27] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
23/12/23 09:01:03 INFO MemoryStore: Block broadcast_7 stored as values in memory (estimated size 14.0 KiB, free 434.3 MiB)
23/12/23 09:01:03 INFO MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 6.9 KiB, free 434.3 MiB)
23/12/23 09:01:03 INFO BlockManagerInfo: Added broadcast_7_piece0 in memory on 193.147.50.16:42587 (size: 6.9 KiB, free: 434.4 

In [7]:
# DF from RDD

# Access the SparkContext object from the SparkSession object
sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Create the DF from the RDD
rdd = sc.parallelize(input_data)
df = rdd.toDF()
df.printSchema()

df.show()

# slide 14

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: long (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)

+---+------------+----------+---+---+-----------+
| _1|          _2|        _3| _4| _5|         _6|
+---+------------+----------+---+---+-----------+
|  1|       Simón|   Bolivar| 47|VEN|489 895 965|
|  2|       Fidel|    Castro| 90| CU|956 268 348|
|  3|        Jose|   Doroteo| 45|MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39| AR|895 325 481|
|  5|        Hugo|    Chávez| 58| VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|CUB|956 268 348|
|  7|    Emiliano|    Zapata| 39| ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|ARG|985 621 444|
+---+------------+----------+---+---+-----------+



In [10]:
# You can pass the column names to the DF

# Access the SparkContext object from the SparkSession object
sc = spark.sparkContext
sc.setLogLevel("ERROR")

# Create the DF from the RDD with column names
rdd = sc.parallelize(input_data)

columns = ["id","name","surname","age","country","local_phone"]
df = rdd.toDF(columns)
df.printSchema()

# slide 14

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- local_phone: string (nullable = true)



In [11]:
# Another method using RDD as input is the createDataFrame method from the
# SparkSession object


rdd = sc.parallelize(input_data)

columns = ["id","name","surname","age","country","local_phone"]
df = spark.createDataFrame(rdd).toDF(*columns)
df.printSchema()
# The toDF method here is different because you are applying it to a DF not to an RDD,
# check specifications on the documentation

# slide 14

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- local_phone: string (nullable = true)



In [12]:
# Using the Row type

rowData = map(lambda x: Row(*x), input_data) 
df = spark.createDataFrame(rowData,columns)
df.printSchema()

# slide 14

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- local_phone: string (nullable = true)



In [13]:
# Creating your schema using python pyspark.sql structures, most consistent approach
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType([ \
    StructField("id", IntegerType(), True), \
    StructField("name",StringType(),True), \
    StructField("surname",StringType(),True), \
    StructField("age", StringType(), True), \
    StructField("country",StringType(),True), \
    StructField("local_phone", StringType(), True), \
  ])

df = spark.createDataFrame(data=input_data,schema=schema)
df.printSchema()
df.show()

# slide 14

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- local_phone: string (nullable = true)

+---+------------+----------+---+-------+-----------+
| id|        name|   surname|age|country|local_phone|
+---+------------+----------+---+-------+-----------+
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|
+---+------------+----------+---+-------+-----------+



In [16]:
# Writing CSV file

output_path2 = "../../data/csv_example/one_file"
output_path1 = "../../data/csv_example/many_files"

# Write the DataFrame to CSV
df.write.mode('overwrite').csv(output_path1, header=True)
df.coalesce(1).write.mode('overwrite').csv(output_path2, header=True)

# slide 14

In [17]:
# Reading CSV file

read_path = '../../data/csv_example/one_file'

print('+ Read csv from file: \n')
df_from_csv = spark.read.csv(read_path)
df_from_csv.printSchema()
df_from_csv.show()

print('+ Read csv with header from file: \n')
df_from_csv = spark.read.csv(read_path,header=True)
df_from_csv.printSchema()
df_from_csv.show()

print('+ Read csv with header and infering data types from file: \n')
df_from_csv = spark.read.csv(read_path,header=True, inferSchema=True)
df_from_csv.printSchema()
df_from_csv.show()

# slide 14

+ Read csv from file: 

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

+---+------------+----------+---+-------+-----------+
|_c0|         _c1|       _c2|_c3|    _c4|        _c5|
+---+------------+----------+---+-------+-----------+
| id|        name|   surname|age|country|local_phone|
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|
+---+------------+----------+---+-------+-----------+

+ Read csv with header from file: 

root
 |-- id: string 

In [18]:
# Saving JSON file
output_path1 = "../../data/json_example/many_files"
output_path2 = "../../data/json_example/one_file"

# Write the DataFrame to CSV
df.write.mode('overwrite').json(output_path1)
df.coalesce(1).write.mode('overwrite').option("multiline","true").json(output_path2)

# slide 14

In [20]:
# Read data from json

read_path = "../../data/json_example/many_files"

print('+ Read json from file: \n')
df_from_json = spark.read.json(read_path)
df_from_json.printSchema()
df_from_json.show()

# slide 14

+ Read json from file: 

root
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- id: long (nullable = true)
 |-- local_phone: string (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)

+---+-------+---+-----------+------------+----------+
|age|country| id|local_phone|        name|   surname|
+---+-------+---+-----------+------------+----------+
| 39|     ME|  7|985 621 444|    Emiliano|    Zapata|
| 78|    ARG|  8|985 621 444|Juan Domingo|     Perón|
| 58|     VE|  5|489 895 965|        Hugo|    Chávez|
| 27|    CUB|  6|956 268 348|      Camilo|Cienfuegos|
| 45|    MEX|  3|985 621 444|        Jose|   Doroteo|
| 39|     AR|  4|895 325 481|     Ernesto|   Guevara|
| 47|    VEN|  1|489 895 965|       Simón|   Bolivar|
| 90|     CU|  2|956 268 348|       Fidel|    Castro|
+---+-------+---+-----------+------------+----------+



In [ ]:
# Write in DB
# You need to have a created DB on a local/remote postgresql server

# Define database connection properties
db_properties = {
    "url": "jdbc:postgresql://localhost:5432/pyspark_db",
    "driver": "org.postgresql.Driver",
    "user": "pyspark",
    "password": "password",
}

table_name = "nice_guys"

# Write the DataFrame to the database
df.write \
    .jdbc(url= db_properties["url"],
          table=table_name,
          mode="overwrite",  # You can use "append" or "ignore" as well
          properties=db_properties)

# slide 14

In [ ]:
# Read DF from databse

# Define database connection properties
db_properties = {
    "url": "jdbc:postgresql://localhost:5432/pyspark_db",
    "driver": "org.postgresql.Driver",
    "user": "pyspark",
    "password": "password",
}

table_name = "nice_guys"

df_from_postgresql = spark.read \
    .jdbc(url=db_properties["url"],
          table="nice_guys",
          properties=db_properties)

df_from_postgresql.show()
df = df_from_postgresql

# slide 14

In [22]:
# Load table with HIVE format to spark catalog to use it as view

warehouse_location = spark.conf.get("spark.sql.warehouse.dir")
table_name = "example_table"

df.write.mode('overwrite').parquet('../../data/parquet_example')

# Load table
df = spark.read.format('parquet').load('../../data/parquet_example')
df.show()

# slide 16

+---+------------+----------+---+-------+-----------+
| id|        name|   surname|age|country|local_phone|
+---+------------+----------+---+-------+-----------+
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|
+---+------------+----------+---+-------+-----------+



In [24]:
# Add table to spark catalog

access_table_name = 'my_table'
df.createOrReplaceTempView(access_table_name)

print('Available tables:')
tables = spark.catalog.listTables()
for table in tables:
    print('+ ',table.name)

# slide 16

Available tables:
+  my_table


In [25]:
# Read some_table if its loaded in the spark.catalog

df = spark.read.table(access_table_name)
df.show()

# slide 16

+---+------------+----------+---+-------+-----------+
| id|        name|   surname|age|country|local_phone|
+---+------------+----------+---+-------+-----------+
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|
+---+------------+----------+---+-------+-----------+



In [26]:
# Use SQL on DF

access_table_name = 'my_table'
result = spark.sql(f"SELECT * FROM {access_table_name} WHERE Age >= 45")
result.show()

# slide 16

+---+------------+-------+---+-------+-----------+
| id|        name|surname|age|country|local_phone|
+---+------------+-------+---+-------+-----------+
|  8|Juan Domingo|  Perón| 78|    ARG|985 621 444|
|  5|        Hugo| Chávez| 58|     VE|489 895 965|
|  3|        Jose|Doroteo| 45|    MEX|985 621 444|
|  1|       Simón|Bolivar| 47|    VEN|489 895 965|
|  2|       Fidel| Castro| 90|     CU|956 268 348|
+---+------------+-------+---+-------+-----------+



In [27]:
# Filter

from pyspark.sql.functions import col

df.filter(col('age') < 45).show()

# slide 17

+---+--------+----------+---+-------+-----------+
| id|    name|   surname|age|country|local_phone|
+---+--------+----------+---+-------+-----------+
|  7|Emiliano|    Zapata| 39|     ME|985 621 444|
|  6|  Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  4| Ernesto|   Guevara| 39|     AR|895 325 481|
+---+--------+----------+---+-------+-----------+



In [28]:
# Order by

df.orderBy('id').show()

df.orderBy('age').show()

# slide 17

+---+------------+----------+---+-------+-----------+
| id|        name|   surname|age|country|local_phone|
+---+------------+----------+---+-------+-----------+
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|
+---+------------+----------+---+-------+-----------+

+---+------------+----------+---+-------+-----------+
| id|        name|   surname|age|country|local_phone|
+---+------------+----------+---+-------+-----------+
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|
|  3|        Jose|   Dorote

In [29]:
# Drop

df.drop('name').show()

# slide 17

+---+----------+---+-------+-----------+
| id|   surname|age|country|local_phone|
+---+----------+---+-------+-----------+
|  7|    Zapata| 39|     ME|985 621 444|
|  8|     Perón| 78|    ARG|985 621 444|
|  5|    Chávez| 58|     VE|489 895 965|
|  6|Cienfuegos| 27|    CUB|956 268 348|
|  3|   Doroteo| 45|    MEX|985 621 444|
|  4|   Guevara| 39|     AR|895 325 481|
|  1|   Bolivar| 47|    VEN|489 895 965|
|  2|    Castro| 90|     CU|956 268 348|
+---+----------+---+-------+-----------+



In [30]:
# Add column to DF

from pyspark.sql.functions import col

new_df = df.withColumn('young', col('age') > 30)
new_df.show()

# SQL equivalent...
access_table_name = 'my_table'
spark.sql(f"SELECT *,(Age > 30) AS young FROM {access_table_name}").show()

# slide 17

+---+------------+----------+---+-------+-----------+-----+
| id|        name|   surname|age|country|local_phone|young|
+---+------------+----------+---+-------+-----------+-----+
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444| true|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444| true|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965| true|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|false|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444| true|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481| true|
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965| true|
|  2|       Fidel|    Castro| 90|     CU|956 268 348| true|
+---+------------+----------+---+-------+-----------+-----+

+---+------------+----------+---+-------+-----------+-----+
| id|        name|   surname|age|country|local_phone|young|
+---+------------+----------+---+-------+-----------+-----+
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444| true|
|  8|Juan Domingo|     Perón| 78|    AR

In [31]:
columns = ["id","name","surname","age","country","local_phone","phone_code"]
input_data = [(1,"Simón","Bolivar",47,"VEN","489 895 965","+58"),
    (2,"Fidel","Castro",90,"CU","956 268 348","+53"),
    (3,"Jose","Doroteo",45,"MEX","985 621 444","+52"),
    (4,"Ernesto","Guevara",39,"AR","895 325 481","+54"),
    (5,"Hugo","Chávez",58,"VE","489 895 965","+58"),
    (6,"Camilo","Cienfuegos",27,"CUB","956 268 348","+53"),
    (7,"Emiliano","Zapata",39,"ME","985 621 444","+52"),
    (8,"Juan Domingo","Perón",78,"ARG","985 621 444","+54"),
  ]

mod_df = spark.createDataFrame(input_data).toDF(*columns)
mod_df.show()

# slide 21

+---+------------+----------+---+-------+-----------+----------+
| id|        name|   surname|age|country|local_phone|phone_code|
+---+------------+----------+---+-------+-----------+----------+
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|       +58|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|       +53|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|       +52|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|       +54|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|       +58|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|       +53|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|       +52|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|       +54|
+---+------------+----------+---+-------+-----------+----------+



In [32]:
# Grouping and Aggregation

from pyspark.sql.functions import count

result = mod_df.groupBy("phone_code") \
                .agg(count("*").alias("item_count"))
result.show()

# slide 21

+----------+----------+
|phone_code|item_count|
+----------+----------+
|       +58|         2|
|       +53|         2|
|       +54|         2|
|       +52|         2|
+----------+----------+



In [33]:
# Grouping and Aggregation 2 columns

from pyspark.sql.functions import collect_list, count

result = mod_df.groupBy("phone_code") \
                .agg(count("*").alias("item_count"), \
                     collect_list("id").alias("ids"))
result.show()

# slide 21

+----------+----------+------+
|phone_code|item_count|   ids|
+----------+----------+------+
|       +58|         2|[1, 5]|
|       +53|         2|[2, 6]|
|       +54|         2|[4, 8]|
|       +52|         2|[3, 7]|
+----------+----------+------+



In [34]:
# Average

from pyspark.sql.functions import avg

result = mod_df.groupBy("phone_code") \
                .agg(avg('age').alias("age_avg"))

result.show()

# slide 21

+----------+-------+
|phone_code|age_avg|
+----------+-------+
|       +58|   52.5|
|       +53|   58.5|
|       +54|   58.5|
|       +52|   42.0|
+----------+-------+



In [35]:
# Collect list, collect set

from pyspark.sql.functions import collect_list, collect_set, col

df = mod_df.withColumn('young', col('age') > 30)
df.show()


result = df.groupBy("young") \
                .agg(
                      collect_list("phone_code").alias("phone_codes"))
result.show(truncate=False)

result = df.groupBy("young") \
                .agg(
                      collect_set("phone_code").alias("phone_codes"))
result.show(truncate=False)


# slide 21

+---+------------+----------+---+-------+-----------+----------+-----+
| id|        name|   surname|age|country|local_phone|phone_code|young|
+---+------------+----------+---+-------+-----------+----------+-----+
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|       +58| true|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|       +53| true|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|       +52| true|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|       +54| true|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|       +58| true|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|       +53|false|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|       +52| true|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|       +54| true|
+---+------------+----------+---+-------+-----------+----------+-----+

+-----+-----------------------------------+
|young|phone_codes                        |
+-----+-----------------------------------+
|true |[+58, +5

In [36]:
# Explode example

from pyspark.sql.functions import explode, collect_list, count

result = mod_df.groupBy("phone_code") \
                .agg(count("*").alias("item_count"), \
                     collect_list("name").alias("names"))
result.show()

result = result.select('phone_code', explode('names').alias('name'))

result.show()


# slide 21

+----------+----------+--------------------+
|phone_code|item_count|               names|
+----------+----------+--------------------+
|       +58|         2|       [Simón, Hugo]|
|       +53|         2|     [Fidel, Camilo]|
|       +54|         2|[Ernesto, Juan Do...|
|       +52|         2|    [Jose, Emiliano]|
+----------+----------+--------------------+

+----------+------------+
|phone_code|        name|
+----------+------------+
|       +58|       Simón|
|       +58|        Hugo|
|       +53|       Fidel|
|       +53|      Camilo|
|       +54|     Ernesto|
|       +54|Juan Domingo|
|       +52|        Jose|
|       +52|    Emiliano|
+----------+------------+



In [37]:
# Timestamps

from pyspark.sql.functions import to_timestamp

df=spark.createDataFrame(
        data = [ ("1","2023-09-23 05:23:02.013")],
        schema=["id","event"])
df.printSchema()

#Timestamp String to DateType
df.withColumn("timestamp",to_timestamp("event")) \
  .show()

# format 'yyyy-MM-dd  HH:mm:ss.SSS'

# Custom format
df=spark.createDataFrame(
        data = [ ("1","23-09-2023 05:23:02.013")],
        schema=["id","event"])
df.printSchema()

#Timestamp String to DateType
df.withColumn("timestamp",to_timestamp("event",'dd-MM-yyyy HH:mm:ss.SSSS')) \
  .show()

# Custom format, be careful things can go wrong
df=spark.createDataFrame(
        data = [ ("1","23-09-2023 05:23:02.013")],
        schema=["id","event"])
df.printSchema()

#Timestamp String to DateType
df.withColumn("timestamp",to_timestamp("event",'MM-dd-yyyy HH:mm:ss.SSSS')) \
  .show()

# slide 21

root
 |-- id: string (nullable = true)
 |-- event: string (nullable = true)

+---+--------------------+--------------------+
| id|               event|           timestamp|
+---+--------------------+--------------------+
|  1|2023-09-23 05:23:...|2023-09-23 05:23:...|
+---+--------------------+--------------------+

root
 |-- id: string (nullable = true)
 |-- event: string (nullable = true)

+---+--------------------+--------------------+
| id|               event|           timestamp|
+---+--------------------+--------------------+
|  1|23-09-2023 05:23:...|2023-09-23 05:23:...|
+---+--------------------+--------------------+

root
 |-- id: string (nullable = true)
 |-- event: string (nullable = true)

+---+--------------------+---------+
| id|               event|timestamp|
+---+--------------------+---------+
|  1|23-09-2023 05:23:...|     NULL|
+---+--------------------+---------+



In [38]:
# Time differences

from pyspark.sql.functions import current_date, datediff, col

data = [("1","2023-09-28"),("2","2001-09-11"),("3","1989-11-09")]
df=spark.createDataFrame(data=data,schema=["id","date"])

df.select(
      col("date"),
      current_date().alias("current_date"),
      datediff(current_date(),col("date")).alias("datediff")
    ).show()

#slide 21

+----------+------------+--------+
|      date|current_date|datediff|
+----------+------------+--------+
|2023-09-28|  2023-12-23|      86|
|2001-09-11|  2023-12-23|    8138|
|1989-11-09|  2023-12-23|   12462|
+----------+------------+--------+



In [39]:
# Sorting

from pyspark.sql.functions import asc_nulls_first

result = mod_df.orderBy(mod_df.age.asc_nulls_first())
result.show()
# You will get null values first

#slide 21

+---+------------+----------+---+-------+-----------+----------+
| id|        name|   surname|age|country|local_phone|phone_code|
+---+------------+----------+---+-------+-----------+----------+
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|       +53|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|       +54|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|       +52|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|       +52|
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|       +58|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|       +58|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|       +54|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|       +53|
+---+------------+----------+---+-------+-----------+----------+



In [40]:
# Using regular expersions

from pyspark.sql.functions import regexp_extract, regexp_replace, concat_ws, upper

prefix_pattern = r'(\d+)'
# Use regexp_extract to extract the prefix from the "local_phone" column
df1 = mod_df.withColumn("phone_code", regexp_extract(col("phone_code"), prefix_pattern, 1))
df1.show()

df2 = mod_df.withColumn("local_phone", regexp_replace(col("local_phone"), r'\s+', ''))
df2.show()

df3 = mod_df.withColumn("phone_number", concat_ws("", regexp_extract(col("phone_code"), prefix_pattern, 1), 
                                                  regexp_replace(col("local_phone"), r'\s+', ''))) \
            .drop('phone_code','local_phone') \
            .withColumn("name", upper(col("name"))) \
            .withColumn("surname", upper(col("surname")))
df3.show()


#slide 21

+---+------------+----------+---+-------+-----------+----------+
| id|        name|   surname|age|country|local_phone|phone_code|
+---+------------+----------+---+-------+-----------+----------+
|  1|       Simón|   Bolivar| 47|    VEN|489 895 965|        58|
|  2|       Fidel|    Castro| 90|     CU|956 268 348|        53|
|  3|        Jose|   Doroteo| 45|    MEX|985 621 444|        52|
|  4|     Ernesto|   Guevara| 39|     AR|895 325 481|        54|
|  5|        Hugo|    Chávez| 58|     VE|489 895 965|        58|
|  6|      Camilo|Cienfuegos| 27|    CUB|956 268 348|        53|
|  7|    Emiliano|    Zapata| 39|     ME|985 621 444|        52|
|  8|Juan Domingo|     Perón| 78|    ARG|985 621 444|        54|
+---+------------+----------+---+-------+-----------+----------+

+---+------------+----------+---+-------+-----------+----------+
| id|        name|   surname|age|country|local_phone|phone_code|
+---+------------+----------+---+-------+-----------+----------+
|  1|       Simón|   Bol

In [41]:
# @udf(returnType=StringType())
def is_even_udf(x):
    return x % 2 == 0

df = spark.range(1, 100).toDF("x")

# is_even_udf = udf(is_even_udf, StringType())

result_df = df.select(col("x"), is_even_udf(col("x")) \
.alias("is_even"))

result_df.printSchema()

#slide 22

root
 |-- x: long (nullable = false)
 |-- is_even: boolean (nullable = true)



In [ ]:
# UDAF, using pandas...

from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType

# Define a custom UDAF to calculate the average age per country
@pandas_udf(DoubleType(), PandasUDFType.GROUPED_AGG)
def average_age_udaf(v):
    return v.mean()

# Use the UDAF to calculate average age per country
result = df.groupBy("country").agg(average_age_udaf(col("age")).alias("average_age"))

# Show the result
result.show()

# slide 23

In [43]:
spark.stop()